# Importing the Debiased Sentences

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

debiased_df_path = "/content/drive/MyDrive/266_Project/debiased_df.csv"
debiased_df = pd.read_csv(debiased_df_path)

## Evaluate Debiasing Performance

### LLAMA Treatment Model

#### 1. Using the RoBERTA classifier

Import the RoBERTA classifier

In [3]:
debiased_df['text'] = debiased_df['text'].astype(str)
debiased_df['debiased_text'] = debiased_df['debiased_text'].astype(str)

In [4]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

saved_model_dir = "/content/drive/MyDrive/266_Project/3.2_RoBERTa_Treatment/roberta_fine_tuned_model"

roberta_tokenizer = RobertaTokenizer.from_pretrained(saved_model_dir)

roberta_model = RobertaForSequenceClassification.from_pretrained(saved_model_dir)

In [5]:
import torch

def classify_sentence(sentence, model, tokenizer, device="cuda:0"):
    model.to(device)
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return prediction

device = "cuda:0" if torch.cuda.is_available() else "cpu"

debiased_df['roberta_debiased_prediction'] = debiased_df['debiased_text'].apply(lambda x: classify_sentence(x, roberta_model, roberta_tokenizer, device=device))

label_mapping = {0: "right", 1: "left", 2: "center"}
debiased_df['roberta_debiased_label'] = debiased_df['roberta_debiased_prediction'].map(label_mapping)

check how many sentences are classified as "center", which would indicate that the LLaMA model successfully neutralized the political bias.

In [6]:
# Calculate the proportion of "center" classifications in the debiased predictions
test_center_ratio = (debiased_df['roberta_debiased_label'] == 'center').mean()
print(f"Proportion of 'center' classifications in debiased test set: {test_center_ratio:.2%}")
debiased_df[['text', 'debiased_text', 'roberta_debiased_label']].head()

Proportion of 'center' classifications in debiased test set: 13.00%


,text,debiased_text,roberta_debiased_label
0,"Indeed, that original NYT report suggested tha...",The original NYT report suggests that the mana...,left
1,How we treat people at or inside our border ce...,The way we treat immigrants is a serious issue...,left
2,The battle of Portland is just plain crazy — b...,The battle of Portland is a fight between prot...,right
3,"As a perennial litigant, Trump weaponized the ...",As an ag,right
4,The president insisted his Department of Healt...,The President said that he believed his depart...,right


In [15]:
# Calculate the proportion of "center" classifications in the original text
test_center_ratio = (debiased_df['type'] == 2).mean()
print(f"Proportion of 'center' classifications in original set: {test_center_ratio:.2%}")

Proportion of 'center' classifications in original set: 0.00%


In [14]:
debiased_df.head()

,text,news_link,label,outlet,topic,type,label_opinion,biased_words,uuid,debiased_text,roberta_debiased_prediction,roberta_debiased_label,original_sentiment,original_sentiment_score,debiased_sentiment,debiased_sentiment_score,original_bias_label,original_bias_score,debiased_bias_label,debiased_bias_score
0,"Indeed, that original NYT report suggested tha...",http://www.msnbc.com/rachel-maddow-show/trumps...,1,MSNBC,immigration,1,Somewhat factual but also opinionated,[],NHcSfAcYxpi6sKDf3gYQuE,The original NYT report suggests that the mana...,1,left,negative,0.599804,negative,0.663784,Biased,0.616854,Non-biased,0.600076
1,How we treat people at or inside our border ce...,https://www.alternet.org/2020/02/how-u-s-forei...,0,Alternet,immigration,1,Somewhat factual but also opinionated,[],6pMhG6BGGmpukBjLYvS7aj,The way we treat immigrants is a serious issue...,1,left,neutral,0.477925,negative,0.639569,Non-biased,0.863805,Biased,0.975347
2,The battle of Portland is just plain crazy — b...,https://www.alternet.org/2020/07/the-battle-of...,1,Alternet,black lives matter,1,Expresses writer’s opinion,"['dystopian', 'crazy']",ak4GUjEPQX4LCZc64iDEVj,The battle of Portland is a fight between prot...,0,right,negative,0.825357,neutral,0.529395,Biased,0.977657,Biased,0.921195
3,"As a perennial litigant, Trump weaponized the ...",https://www.alternet.org/2020/04/sadism-crime-...,1,Alternet,gun control,1,Somewhat factual but also opinionated,"['weaponized', 'abuse', 'disturbing']",JkkkMzPuZpQY2fHJhpuomg,As an ag,0,right,negative,0.673205,neutral,0.732622,Biased,0.945206,Non-biased,0.909679
4,The president insisted his Department of Healt...,https://feeds.feedblitz.com/~/636639870/0/alte...,1,Alternet,universal health care,1,Entirely factual,['claimed'],PD4fzPSyAWgHPwKNGHJHQL,The President said that he believed his depart...,0,right,negative,0.516687,neutral,0.507704,Biased,0.544243,Non-biased,0.592535


Evaluate Performance Metrics

In [7]:
from sklearn.metrics import accuracy_score
true_labels = [2] * len(debiased_df)
test_accuracy = accuracy_score(true_labels, debiased_df['roberta_debiased_prediction'])
print(f"Accuracy: {test_accuracy:.2f}")

Accuracy: 0.13


#### 2. Sentiment Check

In [8]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import urllib.request
import csv

task = "sentiment"
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode("utf-8").split("\n")
    csvreader = csv.reader(html, delimiter="\t")
    labels = [row[1] for row in csvreader if len(row) > 1]

def classify_sentiment(text):
    encoded_input = tokenizer(text, return_tensors="pt")
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # label with the highest score
    ranking = np.argsort(scores)[::-1]
    top_label = labels[ranking[0]]
    top_score = scores[ranking[0]]
    return top_label, top_score

# sentiment analysis to original and debiased text
debiased_df[["original_sentiment", "original_sentiment_score"]] = debiased_df["text"].apply(
    lambda x: pd.Series(classify_sentiment(x))
)
debiased_df[["debiased_sentiment", "debiased_sentiment_score"]] = debiased_df["debiased_text"].apply(
    lambda x: pd.Series(classify_sentiment(x))
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [9]:
# sentiment distribution before and after debiasing
original_sentiment_counts = debiased_df["original_sentiment"].value_counts()
debiased_sentiment_counts = debiased_df["debiased_sentiment"].value_counts()

print("Sentiment Distribution Before Debiasing:")
print(original_sentiment_counts)

print("\nSentiment Distribution After Debiasing:")
print(debiased_sentiment_counts)

# mean sentiment scores before and after debiasing
mean_original_score = debiased_df["original_sentiment_score"].mean()
mean_debiased_score = debiased_df["debiased_sentiment_score"].mean()

print(f"\nMean Sentiment Score Before Debiasing: {mean_original_score:.4f}")
print(f"Mean Sentiment Score After Debiasing: {mean_debiased_score:.4f}")

Sentiment Distribution Before Debiasing:
original_sentiment
negative    59
neutral     38
positive     3
Name: count, dtype: int64

Sentiment Distribution After Debiasing:
debiased_sentiment
negative    49
neutral     44
positive     7
Name: count, dtype: int64

Mean Sentiment Score Before Debiasing: 0.6974
Mean Sentiment Score After Debiasing: 0.7023


#### 3. D4 Data Bias Detection Model

In [10]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")
bias_classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [11]:
def classify_bias(sentence):
    if isinstance(sentence, str) and sentence.strip():
        result = bias_classifier(sentence)
        return result[0]['label'], result[0]['score']
    else:
        return "Invalid", 0.0

# classification of original and debiased sentences
debiased_df[['original_bias_label', 'original_bias_score']] = debiased_df['text'].apply(
    lambda x: pd.Series(classify_bias(x))
)
debiased_df[['debiased_bias_label', 'debiased_bias_score']] = debiased_df['debiased_text'].apply(
    lambda x: pd.Series(classify_bias(x))
)

debiased_df.head()

,text,news_link,label,outlet,topic,type,label_opinion,biased_words,uuid,debiased_text,roberta_debiased_prediction,roberta_debiased_label,original_sentiment,original_sentiment_score,debiased_sentiment,debiased_sentiment_score,original_bias_label,original_bias_score,debiased_bias_label,debiased_bias_score
0,"Indeed, that original NYT report suggested tha...",http://www.msnbc.com/rachel-maddow-show/trumps...,1,MSNBC,immigration,1,Somewhat factual but also opinionated,[],NHcSfAcYxpi6sKDf3gYQuE,The original NYT report suggests that the mana...,1,left,negative,0.599804,negative,0.663784,Biased,0.616854,Non-biased,0.600076
1,How we treat people at or inside our border ce...,https://www.alternet.org/2020/02/how-u-s-forei...,0,Alternet,immigration,1,Somewhat factual but also opinionated,[],6pMhG6BGGmpukBjLYvS7aj,The way we treat immigrants is a serious issue...,1,left,neutral,0.477925,negative,0.639569,Non-biased,0.863805,Biased,0.975347
2,The battle of Portland is just plain crazy — b...,https://www.alternet.org/2020/07/the-battle-of...,1,Alternet,black lives matter,1,Expresses writer’s opinion,"['dystopian', 'crazy']",ak4GUjEPQX4LCZc64iDEVj,The battle of Portland is a fight between prot...,0,right,negative,0.825357,neutral,0.529395,Biased,0.977657,Biased,0.921195
3,"As a perennial litigant, Trump weaponized the ...",https://www.alternet.org/2020/04/sadism-crime-...,1,Alternet,gun control,1,Somewhat factual but also opinionated,"['weaponized', 'abuse', 'disturbing']",JkkkMzPuZpQY2fHJhpuomg,As an ag,0,right,negative,0.673205,neutral,0.732622,Biased,0.945206,Non-biased,0.909679
4,The president insisted his Department of Healt...,https://feeds.feedblitz.com/~/636639870/0/alte...,1,Alternet,universal health care,1,Entirely factual,['claimed'],PD4fzPSyAWgHPwKNGHJHQL,The President said that he believed his depart...,0,right,negative,0.516687,neutral,0.507704,Biased,0.544243,Non-biased,0.592535


In [12]:
original_bias_counts = debiased_df['original_bias_label'].value_counts()

debiased_bias_counts = debiased_df['debiased_bias_label'].value_counts()

print("Original Text Bias Counts:")
print(original_bias_counts)

print("\nDebiased Text Bias Counts:")
print(debiased_bias_counts)

Original Text Bias Counts:
original_bias_label
Biased        57
Non-biased    43
Name: count, dtype: int64

Debiased Text Bias Counts:
debiased_bias_label
Non-biased    50
Biased        50
Name: count, dtype: int64


In [13]:
output_path = "/content/drive/MyDrive/266_Project/debiased_df_with_performance.csv"
debiased_df.to_csv(output_path, index=False)